# Solution for Assignment 4: Effect of Demand Distribution on Order Quantity

This notebook solves the problems from Assignment 4 using the `imf_*` formula library to explore how different demand distributions affect the optimal order quantity in the newsvendor model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from scipy.stats import uniform, poisson, gamma

# Get the absolute path of the current file and add parent directory to path
current_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(current_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

# Now we can import from imf_main
try:
    from imf_main import (
        newsvendor_critical_ratio,
        newsvendor_uniform,
        newsvendor_poisson,
        newsvendor_gamma
    )
except ImportError:
    print("Could not import from imf_main. Using direct implementation...")
    
    # Direct implementation of newsvendor functions if import fails
    def newsvendor_critical_ratio(price, cost, salvage=0):
        """Calculate newsvendor critical ratio"""
        return (price - cost) / (price - salvage)
    
    def newsvendor_uniform(lower_bound, upper_bound, critical_ratio):
        """Calculate newsvendor order quantity for uniformly distributed demand"""
        return lower_bound + critical_ratio * (upper_bound - lower_bound)
    
    def newsvendor_poisson(lambda_param, critical_ratio):
        """Calculate newsvendor order quantity for Poisson distributed demand"""
        return poisson.ppf(critical_ratio, lambda_param)
    
    def newsvendor_gamma(mean, std, critical_ratio):
        """Calculate newsvendor order quantity for gamma distributed demand"""
        var = std**2
        alpha = mean**2 / var  # shape parameter
        beta = var / mean      # scale parameter
        return gamma.ppf(critical_ratio, a=alpha, scale=beta)

## Critical Ratio

We'll use the same critical ratio for all distributions to facilitate comparison.

In [ ]:
# Critical ratio for all distributions
critical_ratio = 0.75
print(f"Critical ratio: {critical_ratio}")

## Exercise 4(a): Uniform Distribution

In [ ]:
# Uniform distribution parameters
lower_bound = 4000
upper_bound = 8000

# Calculate optimal order quantity using the library function
optimal_q_uniform = newsvendor_uniform(lower_bound, upper_bound, critical_ratio)
print(f"Order quantity with uniformly distributed demand: {optimal_q_uniform:.0f} units")

# Calculate mean and variance for uniform distribution
uniform_mean = (lower_bound + upper_bound) / 2
uniform_std = (upper_bound - lower_bound) / np.sqrt(12)
print(f"Uniform mean: {uniform_mean:.0f}")
print(f"Uniform standard deviation: {uniform_std:.2f}")

# Plot the uniform PDF
x = np.linspace(lower_bound - 500, upper_bound + 500, 1000)
y = uniform.pdf(x, loc=lower_bound, scale=upper_bound-lower_bound)

plt.figure(figsize=(10, 6))
plt.plot(x, y, 'g-', linewidth=2)
plt.axvline(x=optimal_q_uniform, color='r', linestyle='--', label=f'Optimal order quantity: {optimal_q_uniform:.0f}')
plt.axvline(x=uniform_mean, color='b', linestyle=':', label=f'Mean demand: {uniform_mean:.0f}')

# Fill areas to show overage and underage regions
plt.fill_between(x, y, where=(x <= optimal_q_uniform), alpha=0.3, color='green', label='No stockout')
plt.fill_between(x, y, where=(x > optimal_q_uniform), alpha=0.3, color='red', label='Stockout')

plt.xlabel('Demand')
plt.ylabel('Probability Density')
plt.title('Uniform Demand Distribution')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## Exercise 4(b): Poisson Distribution

In [ ]:
# Poisson distribution parameter
lambda_param = 6000

# Calculate optimal order quantity using the library function
optimal_q_poisson = newsvendor_poisson(lambda_param, critical_ratio)
print(f"Order quantity with Poisson demand: {optimal_q_poisson:.0f} units")

# Calculate mean and variance for Poisson distribution
poisson_mean = lambda_param
poisson_std = np.sqrt(lambda_param)
print(f"Poisson mean: {poisson_mean:.0f}")
print(f"Poisson standard deviation: {poisson_std:.2f}")

# Plot the Poisson PMF
# For Poisson with large lambda, we can approximate the range
x_range = np.arange(lambda_param - 4*poisson_std, lambda_param + 4*poisson_std, 10)
pmf_values = poisson.pmf(x_range, lambda_param)

plt.figure(figsize=(10, 6))
plt.vlines(x_range, 0, pmf_values, colors='b', lw=2, alpha=0.7)
plt.axvline(x=optimal_q_poisson, color='r', linestyle='--', label=f'Optimal order quantity: {optimal_q_poisson:.0f}')
plt.axvline(x=poisson_mean, color='b', linestyle=':', label=f'Mean demand: {poisson_mean:.0f}')

# Mark stockout and no stockout regions
for i, x in enumerate(x_range):
    if x <= optimal_q_poisson:
        plt.vlines(x, 0, pmf_values[i], colors='g', lw=2, alpha=0.5)
    else:
        plt.vlines(x, 0, pmf_values[i], colors='r', lw=2, alpha=0.5)

plt.xlabel('Demand')
plt.ylabel('Probability Mass')
plt.title('Poisson Demand Distribution')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## Exercise 4(c): Gamma Distribution

In [ ]:
# Gamma distribution parameters
gamma_mean = 6000
gamma_std = uniform_std  # Using same std as uniform for comparison

# Calculate optimal order quantity using the library function
optimal_q_gamma = newsvendor_gamma(gamma_mean, gamma_std, critical_ratio)
print(f"Order quantity with Gamma demand: {optimal_q_gamma:.0f} units")

# Calculate shape and scale parameters for gamma distribution
gamma_var = gamma_std**2
gamma_shape = gamma_mean**2 / gamma_var
gamma_scale = gamma_var / gamma_mean
print(f"Gamma shape parameter (α): {gamma_shape:.2f}")
print(f"Gamma scale parameter (β): {gamma_scale:.2f}")

# Plot the gamma PDF
x = np.linspace(gamma.ppf(0.001, gamma_shape, scale=gamma_scale),
                gamma.ppf(0.999, gamma_shape, scale=gamma_scale), 1000)
y = gamma.pdf(x, gamma_shape, scale=gamma_scale)

plt.figure(figsize=(10, 6))
plt.plot(x, y, 'r-', linewidth=2)
plt.axvline(x=optimal_q_gamma, color='r', linestyle='--', label=f'Optimal order quantity: {optimal_q_gamma:.0f}')
plt.axvline(x=gamma_mean, color='b', linestyle=':', label=f'Mean demand: {gamma_mean:.0f}')

# Fill areas to show overage and underage regions
plt.fill_between(x, y, where=(x <= optimal_q_gamma), alpha=0.3, color='green', label='No stockout')
plt.fill_between(x, y, where=(x > optimal_q_gamma), alpha=0.3, color='red', label='Stockout')

plt.xlabel('Demand')
plt.ylabel('Probability Density')
plt.title('Gamma Demand Distribution')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## Comparing Distributions

In [ ]:
# Create plot for all three distributions
fig, axs = plt.subplots(3, 1, figsize=(10, 12))

# (a) Uniform distribution
x_uni = np.linspace(lower_bound - 500, upper_bound + 500, 1000)
pdf_uni = uniform.pdf(x_uni, loc=lower_bound, scale=upper_bound-lower_bound)
axs[0].plot(x_uni, pdf_uni, 'g-', label='Uniform PDF')
axs[0].axvline(x=optimal_q_uniform, color='r', linestyle='--', label=f'Order quantity: {optimal_q_uniform:.0f}')
axs[0].axvline(x=uniform_mean, color='b', linestyle=':', label='Mean demand')
axs[0].set_title(f'Uniform Distribution ({lower_bound}-{upper_bound})')
axs[0].set_xlabel('Demand')
axs[0].set_ylabel('Probability Density')
axs[0].legend()
axs[0].grid(True, alpha=0.3)

# (b) Poisson distribution
x_poisson = np.arange(poisson.ppf(0.001, lambda_param), poisson.ppf(0.999, lambda_param), 10)
poisson_pmf = poisson.pmf(x_poisson, lambda_param)
axs[1].vlines(x_poisson, 0, poisson_pmf, colors='b', lw=2, alpha=0.7, label='Poisson PMF')
axs[1].axvline(x=optimal_q_poisson, color='r', linestyle='--', label=f'Order quantity: {optimal_q_poisson:.0f}')
axs[1].axvline(x=poisson_mean, color='b', linestyle=':', label='Mean demand')
axs[1].set_title(f'Poisson Distribution (λ = {lambda_param})')
axs[1].set_xlabel('Demand')
axs[1].set_ylabel('Probability Mass')
axs[1].legend()
axs[1].grid(True, alpha=0.3)

# (c) Gamma distribution
x_gamma = np.linspace(gamma.ppf(0.001, gamma_shape, scale=gamma_scale),
                      gamma.ppf(0.999, gamma_shape, scale=gamma_scale), 1000)
gamma_pdf = gamma.pdf(x_gamma, gamma_shape, scale=gamma_scale)
axs[2].plot(x_gamma, gamma_pdf, 'r-', label='Gamma PDF')
axs[2].axvline(x=optimal_q_gamma, color='r', linestyle='--', label=f'Order quantity: {optimal_q_gamma:.0f}')
axs[2].axvline(x=gamma_mean, color='b', linestyle=':', label='Mean demand')
axs[2].set_title(f'Gamma Distribution (α = {gamma_shape:.2f}, β = {gamma_scale:.2f})')
axs[2].set_xlabel('Demand')
axs[2].set_ylabel('Probability Density')
axs[2].legend()
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle(f'Effect of Demand Distribution on Order Quantity (CR = {critical_ratio})', fontsize=16, y=1.02)
plt.show()

## Comparison of Mean Demand and Optimal Order Quantity

In [ ]:
# Calculate means and order quantities for comparison
means = [uniform_mean, poisson_mean, gamma_mean]
order_quantities = [optimal_q_uniform, optimal_q_poisson, optimal_q_gamma]

# Calculate differences between order quantity and mean
differences = [order_quantities[i] - means[i] for i in range(3)]

# Create grouped bar chart
bar_width = 0.35
index = np.arange(3)

plt.figure(figsize=(12, 7))
plt.bar(index - bar_width/2, means, bar_width, label='Mean Demand', color='blue', alpha=0.6)
plt.bar(index + bar_width/2, order_quantities, bar_width, label='Optimal Order Quantity', color='red', alpha=0.6)

# Add labels
plt.xlabel('Demand Distribution', fontsize=12)
plt.ylabel('Units', fontsize=12)
plt.title('Comparison of Mean Demand and Optimal Order Quantity Across Distributions', fontsize=14)
plt.xticks(index, ('Uniform', 'Poisson', 'Gamma'), fontsize=12)
plt.yticks(fontsize=10)

# Add text labels for differences
for i, diff in enumerate(differences):
    plt.text(index[i] + bar_width/2, order_quantities[i] + 100, 
             f"+{diff:.0f}", 
             ha='center', va='bottom',
             color='black', fontsize=10, fontweight='bold')

# Add exact values on top of bars
for i, v in enumerate(means):
    plt.text(index[i] - bar_width/2, v + 100, f"{v:.0f}", ha='center', va='bottom', fontsize=10)
for i, v in enumerate(order_quantities):
    plt.text(index[i] + bar_width/2, v + 100, f"{v:.0f}", ha='center', va='bottom', fontsize=10)

plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Analysis of Results

We can see that different demand distributions lead to different optimal order quantities, even when they have similar means and the critical ratio is the same:

1. **Uniform Distribution**: The optimal order quantity is exactly at the critical ratio point in the distribution, which is expected due to the uniform nature of the distribution.

2. **Poisson Distribution**: The optimal order quantity is above the mean, accounting for the discrete nature of the Poisson distribution and its skewness.

3. **Gamma Distribution**: The optimal order quantity is also above the mean, reflecting the right-skewed nature of the gamma distribution.

This demonstrates that the shape of the demand distribution has a significant effect on the optimal inventory policy, and simply using the mean demand without considering the distribution can lead to suboptimal decisions.